In [0]:
%sql
select 
  count(order_id) as total_orders, 
  (case 
     when order_dow = '0' then 'Sunday'
     when order_dow = '1' then 'Monday'
     when order_dow = '2' then 'Tuesday'
     when order_dow = '3' then 'Wednesday'
     when order_dow = '4' then 'Thursday'
     when order_dow = '5' then 'Friday'
     when order_dow = '6' then 'Saturday'              
   end) as day_of_week 
  from workspace.instacart.orders
 group by order_dow 
 order by total_orders desc

In [0]:
%sql
select 
  count(order_id) as total_orders, 
  order_hour_of_day as hour 
  from workspace.instacart.orders 
 group by order_hour_of_day 
 order by order_hour_of_day

In [0]:
%sql
select 
  count(order_id) as total_orders, 
  order_hour_of_day as hour 
  from workspace.instacart.orders
 group by order_hour_of_day 
 order by order_hour_of_day

In [0]:
from pyspark.sql.functions import collect_set

rawData = spark.sql(
    """
    SELECT pr.order_id, p.product_id
    FROM workspace.instacart.order_products_prior pr
    INNER JOIN workspace.instacart.order_products_train p
        ON pr.product_id = p.product_id
    """
)

baskets = rawData.groupBy('order_id').agg(
    collect_set('product_id').alias('items')
)
baskets.createOrReplaceTempView('baskets')

In [0]:
# Visualize the distribution of basket sizes
from pyspark.sql.functions import size

basket_sizes = baskets.withColumn("basket_size", size("items"))
display(basket_sizes.select("basket_size"))

# Visualize the most common products
top_products = spark.sql("""
    SELECT product_name, COUNT(*) as count
    FROM products p
    JOIN order_products_train o ON p.product_id = o.product_id
    GROUP BY product_name
    ORDER BY count DESC
    LIMIT 20
""")
display(top_products)

# Visualize the number of orders per user
orders_per_user = orders.groupBy("user_id").agg(count("order_id").alias("num_orders"))
display(orders_per_user.select("num_orders"))

In [0]:
display(
  spark.sql(
    """
    SELECT *
    FROM orders
    LIMIT 10
    """
  )
)